# Greater Pittsburgh Community Food Bank ingestion script prototype

This notebook is intended to prototype code snippets for an ingestion script aimed at "2019-10-10 PGH Food Bank Site Addresses.xlsx".

Dependencies:
* pandas
* xlrd
* os

In [1]:
import pandas as pd
import os

In [2]:
in_path = '../food-data/PFPC_data_files/2019-10-10 PGH Food Bank Site Addresses.xlsx'
merged_path = '../merged_datasets.csv'

In [4]:
df = pd.read_excel(in_path)
df

,Food Bank - Get Help,Food Bank - Inactive In Inventory System,AgencyRef,AgencyName,GroupTypeOne,GroupTypeTwo,GroupTypeThree,Addr1,Addr2,City,State,Zip,County,County.1,Google Long,Google Lat
0,NaN,0.0,PAGI2450-1,A Giving Heart,Prepared Meals,On Site Other,Childrens-After School,816 Climax Street,NaN,Pittsburgh,PA,15210,Allegheny,Allegheny County,-79.9928,40.4198
1,NaN,0.0,PADA2545-1,Adagio Health,Packaged Food,Health and Wellness Partner,Health and Wellness Partner,116 Browns Hill Road,NaN,Valencia,PA,16059,Allegheny,Allegheny County,-79.9294,40.7135
2,NaN,0.0,PADE2513-1,Adelphoi Education Millvale,Packaged Food,Backpack,BACKPACK,608 Farragut Street,NaN,Pittsburgh,PA,15209,Allegheny,Allegheny County,-79.976,40.4801
3,NaN,0.0,PADO2493-1,Adolecent Medicine UPMC,Packaged Food,Health and Wellness Partner,Health and Wellness Partner,4401 Penn Ave.,Floor 3,Pittsburgh,PA,15224,Allegheny,Allegheny County,-79.9531,40.4676
4,NaN,0.0,PAGH2543-1,AGH Federal North,Packaged Food,Health and Wellness Partner,Health and Wellness Partner,1307 Federal Street,NaN,Pittsburgh,PA,15212,Allegheny,Allegheny County,-80.0078,40.4572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,NaN,NaN,PDO-008,Worthington Area Food Bank,Mixed Programs,PDO Agency,"Other Food Bank, PDO sub-pantry",161 Lairds Crossing Road,NaN,Worthington,PA,16262,Armstrong,Armstrong County,-79.6265,40.853
780,NaN,NaN,PDO-073,Youghiogheny Western Baptist Association,Mixed Programs,PDO Agency,"Other Food Bank, PDO sub-pantry",4431 Morgantown Rd.,NaN,Point Marion,PA,15401,Fayette,Fayette County,-79.8454,39.7616
781,NaN,NaN,PDO-101,Young Township,Mixed Programs,PDO Agency,"Other Food Bank, PDO sub-pantry",1790 Iselin Road,NaN,Faltsburg,PA,15681,Indiana,Indiana County,-79.3861,40.5599
782,NaN,NaN,PDO-175,ywca,Mixed Programs,PDO Agency,"Other Food Bank, PDO sub-pantry",PO Box 364,NaN,Masontown,PA,15461,Fayette,Fayette County,-79.8998,39.8467


In [17]:
# Drop that one empty row at the end
df = df[df['AgencyRef'].notna()]

# Keep only active sites
df = df[df['Food Bank - Inactive In Inventory System'] == 0] # what about all the blanks?

# Assign some columns to schema fields
df['original_id'] = df['AgencyRef']
df['name'] = df['AgencyName']


# Set some fields directly
df['source_org'] = 'Greater Pittsburgh Community Food Bank'
df['source_file'] = os.path.basename(in_path)
df['type'] = 'food bank site'
df['latlng_source'] = df['source_org']
df['free_distribution'] = 1

# Set the fresh_produce flag
df['GroupTypes'] = df['GroupTypeOne'] + df['GroupTypeTwo'] + df['GroupTypeThree']
df['fresh_produce'] = 0
df.loc[df['GroupTypes'].str.contains('Grocery') | df['GroupTypes'].str.contains('Fresh Market'), 'fresh_produce'] = 1

# Clean up and concatenate address fields
df['Addr1'] = df['Addr1'].str.replace('  ', ' ').str.strip(' ')
df['Addr2'] = df['Addr2'].str.replace('  ', ' ').str.strip(' ')
df.loc[df['Addr2'].notna(), 'address'] = df['Addr1'] + ', ' + df['Addr2']
df.loc[df['Addr2'].isna(), 'address'] = df['Addr1']

,Food Bank - Get Help,Food Bank - Inactive In Inventory System,AgencyRef,AgencyName,GroupTypeOne,GroupTypeTwo,GroupTypeThree,Addr1,Addr2,City,State,Zip,County,County.1,Google Long,Google Lat
0,NaN,0.0,PAGI2450-1,A Giving Heart,Prepared Meals,On Site Other,Childrens-After School,816 Climax Street,NaN,Pittsburgh,PA,15210,Allegheny,Allegheny County,-79.9928,40.4198
1,NaN,0.0,PADA2545-1,Adagio Health,Packaged Food,Health and Wellness Partner,Health and Wellness Partner,116 Browns Hill Road,NaN,Valencia,PA,16059,Allegheny,Allegheny County,-79.9294,40.7135
2,NaN,0.0,PADE2513-1,Adelphoi Education Millvale,Packaged Food,Backpack,BACKPACK,608 Farragut Street,NaN,Pittsburgh,PA,15209,Allegheny,Allegheny County,-79.976,40.4801
3,NaN,0.0,PADO2493-1,Adolecent Medicine UPMC,Packaged Food,Health and Wellness Partner,Health and Wellness Partner,4401 Penn Ave.,Floor 3,Pittsburgh,PA,15224,Allegheny,Allegheny County,-79.9531,40.4676
4,NaN,0.0,PAGH2543-1,AGH Federal North,Packaged Food,Health and Wellness Partner,Health and Wellness Partner,1307 Federal Street,NaN,Pittsburgh,PA,15212,Allegheny,Allegheny County,-80.0078,40.4572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,NaN,0.0,PWOM1387-1,"Womanspace East, Inc.",Prepared Meals,On Site Other,Shelter-Battered Women,NaN,NaN,NaN,NaN,NaN,Allegheny,Allegheny County,0,0
556,NaN,0.0,PWOM1386-1,Women's Center and Shelter of Greater Pittsburgh,Prepared Meals,On Site Other,Shelter-Battered Women,NaN,NaN,NaN,NaN,NaN,Allegheny,Allegheny County,0,0
557,NaN,0.0,PYMC1391-1,YMCA New Kensington,Prepared Meals,On Site Other,Childrens- Day Care,800 Constitution Boulevard,NaN,New Kensington,PA,15068,Allegheny,Allegheny County,-79.7634,40.5646
558,NaN,0.0,PYOR2318-1,York Commons,Packaged Food,Senior Box Distribution Site,Senior Box Distribution Site,4003 Penn Ave.,NaN,Pittsburgh,PA,15224,Allegheny,Allegheny County,-79.9575,40.4656


,Food Bank - Get Help,Food Bank - Inactive In Inventory System,AgencyRef,AgencyName,GroupTypeOne,GroupTypeTwo,GroupTypeThree,Addr1,Addr2,City,State,Zip,County,County.1,Google Long,Google Lat,GroupTypes,address
0,NaN,0.0,PAGI2450-1,A Giving Heart,Prepared Meals,On Site Other,Childrens-After School,816 Climax Street,NaN,Pittsburgh,PA,15210,Allegheny,Allegheny County,-79.9928,40.4198,Prepared MealsOn Site OtherChildrens-After School,816 Climax Street
1,NaN,0.0,PADA2545-1,Adagio Health,Packaged Food,Health and Wellness Partner,Health and Wellness Partner,116 Browns Hill Road,NaN,Valencia,PA,16059,Allegheny,Allegheny County,-79.9294,40.7135,Packaged FoodHealth and Wellness PartnerHealth...,116 Browns Hill Road
2,NaN,0.0,PADE2513-1,Adelphoi Education Millvale,Packaged Food,Backpack,BACKPACK,608 Farragut Street,NaN,Pittsburgh,PA,15209,Allegheny,Allegheny County,-79.976,40.4801,Packaged FoodBackpackBACKPACK,608 Farragut Street
3,NaN,0.0,PADO2493-1,Adolecent Medicine UPMC,Packaged Food,Health and Wellness Partner,Health and Wellness Partner,4401 Penn Ave.,Floor 3,Pittsburgh,PA,15224,Allegheny,Allegheny County,-79.9531,40.4676,Packaged FoodHealth and Wellness PartnerHealth...,"4401 Penn Ave., Floor 3"
4,NaN,0.0,PAGH2543-1,AGH Federal North,Packaged Food,Health and Wellness Partner,Health and Wellness Partner,1307 Federal Street,NaN,Pittsburgh,PA,15212,Allegheny,Allegheny County,-80.0078,40.4572,Packaged FoodHealth and Wellness PartnerHealth...,1307 Federal Street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,NaN,NaN,PDO-178,Whitely-Perry Pantry,Mixed Programs,PDO Agency,"Other Food Bank, PDO sub-pantry",P.O. Box 409,NaN,Mt. Morris,PA,15349,Greene,Greene County,-80.0678,39.7331,"Mixed ProgramsPDO AgencyOther Food Bank, PDO s...",P.O. Box 409
779,NaN,NaN,PDO-008,Worthington Area Food Bank,Mixed Programs,PDO Agency,"Other Food Bank, PDO sub-pantry",161 Lairds Crossing Road,NaN,Worthington,PA,16262,Armstrong,Armstrong County,-79.6265,40.853,"Mixed ProgramsPDO AgencyOther Food Bank, PDO s...",161 Lairds Crossing Road
780,NaN,NaN,PDO-073,Youghiogheny Western Baptist Association,Mixed Programs,PDO Agency,"Other Food Bank, PDO sub-pantry",4431 Morgantown Rd.,NaN,Point Marion,PA,15401,Fayette,Fayette County,-79.8454,39.7616,"Mixed ProgramsPDO AgencyOther Food Bank, PDO s...",4431 Morgantown Rd.
781,NaN,NaN,PDO-101,Young Township,Mixed Programs,PDO Agency,"Other Food Bank, PDO sub-pantry",1790 Iselin Road,NaN,Faltsburg,PA,15681,Indiana,Indiana County,-79.3861,40.5599,"Mixed ProgramsPDO AgencyOther Food Bank, PDO s...",1790 Iselin Road


In [9]:
final_cols = ['id', 'source_org', 'source_file', 'original_id', 'type', 'name', 'address', 'city', 
              'state', 'zip_code', 'county', 'location_description', 'phone', 'url', 'latitude', 
              'longitude', 'latlng_source', 'date_from', 'date_to', 'SNAP', 'WIC', 'FMNP', 
              'fresh_produce', 'food_bucks', 'free_distribution', 'open_to_spec_group']

In [11]:
df.reindex(columns = final_cols)

,id,source_org,source_file,original_id,type,name,address,city,state,zip_code,...,latlng_source,date_from,date_to,SNAP,WIC,FMNP,fresh_produce,food_bucks,free_distribution,open_to_spec_group
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
782,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
